In [1]:
import torch
import sys
import os
import numpy as np
import pandas as pd
sys.path.append(os.path.abspath("../..")) 
from datetime import datetime

if torch.cuda.is_available():
    device = torch.device('cuda')
    print("GPU name:", torch.cuda.get_device_name(0))
else:
    device = torch.device('cpu')
print(f'Using device: {device}')

from dataloader.dataloader import load_MNIST_data,load_CINIC10_data,load_CIFAR10_data,load_Fmnist_data,load_Covtype_data
from train.training import train_nomal,train_for_DEQ
from train.evaluate import plot_loss_curve,plot_errorbar_losscurve,plot_confusion_matrix,plot_histograms,create_table,convergence_verify
from result_management.data_manager import save_csv,auto_git_push,save_experiment_report,create_result_pdf
now = datetime.now()
formatted_time = now.strftime("%m%d%H%M")
formatted_time = int(formatted_time)
print(f'-----Formatted time: {formatted_time} -----')

GPU name: NVIDIA GeForce GTX 1660
Using device: cuda
-----Formatted time: 8200928 -----


In [ ]:
experiment_type = "Normal"
experiment_name = f"{experiment_type}{formatted_time}"
variable_param = "leverage" #ここで設定した項目は配列にすること(none,leverage,alpha)
save = True

params = {
    'none':[0], #variable_param=noneの際は1回だけ繰り返す
    #data---------------------------------------------
    'dataset': 'covtype', # 'mnist', 'cifar-10', 'cinic-10' , 'fashion-mnist'
    'batch_size': 1024, #64 MNIST, 100 CIFAR10, 100 CINIC10

    #Encoder_Model--------------------------------
    'enc_type': 'PM', # 'none', 'MZM', 'LI'
    'alpha': np.pi/2, 
    #位相変調機の感度[np.pi*2,np.pi, np.pi/2, np.pi/4, np.pi/8, np.pi/16],pi:-π~π
    #class_model--------------------------------------
    'cls_type': 'MLP', # 'MLP' or 'CNN'
    'num_layer': 3,
    'fc': 'relu',
    'dropout': 0.0,

    #learning-----------------------------------------
    'loss_func': 'cross_entropy',
    'optimizer': 'adam',
    'lr': 3e-4,

    #param--------------------------------------------
    'num_try': 3,
    'max_epochs': 50,
    'leverage': [2,6,9,18,27,54], #mnist:[1,2,4,8,16],covtype:[2,6,9,18,27,54] enc is not none
    'kernel_size': 4
}
#save---------------------------------------------
folder_params = {k: params[k] for k in ['dataset', 'enc_type', 'cls_type']}
if save:
    save_experiment_report(variable_param, params,experiment_name=experiment_name)

data_loaders = {
    'cifar-10': load_CIFAR10_data,
    'cinic-10': load_CINIC10_data,
    'mnist': load_MNIST_data,
    'fashion-mnist':load_Fmnist_data,
    'covtype': load_Covtype_data
}

data_train,data_test = data_loaders[params["dataset"]]()
if params["enc_type"] == 'none':
    leverage = 1
results = []
All_last_LOSSs_ = []
All_last_ACCs_ = []
All_TIMEs_ = []

for variable in params[variable_param]: #variable:leverage,alpha
    print(f'----------------------Running with {variable_param}: {variable}----------------------')
    All_last_loss = []
    All_loss_test = []
    All_pro_time = []
    All_test_acc = []

    for num_times in range(params['num_try']):
        params_for_train = {k: v for k,v in params.items() if k not in ('none',variable_param)}#配列を除外

        if variable_param != 'none': #leverageやalpha可変のとき
            params_for_train.update({'num_times': num_times, variable_param: variable,'device': device})
        else: #パラメータ不変のとき
            params_for_train.update({'num_times': num_times,'device': device}) 
        
        #-----------training-----------
        loss_train_,loss_test_,pro_time_,Last_loss_test,Test_acc,all_labels,all_preds = train_nomal(**params_for_train,data_train=data_train,data_test=data_test)

        All_loss_test.append(loss_test_)
        All_pro_time.append(sum(pro_time_))
        All_last_loss.append(Last_loss_test)
        All_test_acc.append(Test_acc)
        if save:
            datas = [loss_train_,loss_test_,all_labels,all_preds,Test_acc]
            save_csv(datas,variable_param,variable,num_times,**folder_params,save_type='trial',experiment_name=experiment_name)
        print(f"Test Accuracy:{Test_acc:.2f}")
        plot_loss_curve(loss_train_,loss_test_,Save=False,Show=True)
        plot_confusion_matrix(all_labels,all_preds,params["dataset"],Test_acc,Save=False,
                              Show=True)

    datas = [All_loss_test,All_test_acc,All_last_loss,All_pro_time]

    if save:
        save_csv(datas,variable_param,variable,num_times,**folder_params,save_type='mid',experiment_name=experiment_name)

    # plot_errorbar_losscurve(All_loss_test)
    # create_table(All_test_acc,All_last_loss,All_pro_time)

    All_last_ACCs_.append(All_test_acc)
    All_last_LOSSs_.append(All_last_loss)
    All_TIMEs_.append(All_pro_time)

if variable_param != 'none'and save:
    datas = [All_last_ACCs_,All_last_LOSSs_,All_TIMEs_]
    save_csv(datas,variable_param,variable,num_times,**folder_params,save_type='final',experiment_name=experiment_name) #最終保存

create_result_pdf(variable_param, params,experiment_name=experiment_name)

実験パラメータ報告書を保存しました: C:\Users\Scent\OneDrive\PhotonicEncoder_data\covtype\leverage_variable\PM\MLP\Normal8200928\experiment_report.txt
----------------------Running with leverage: 2----------------------


1/3th Epoch:14/50(100.00%) 